In [52]:
import pandas as pd
import numpy as np # type: ignore
from pybaseball import *
import statsapi
import os

from pull_data import *
%load_ext autoreload
%autoreload 2
import requests

from pybaseball import cache
cache.enable()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

team_ids = [138, 140, 145, 110, 111, 144, 158, 143, 133, 141, 108, 114, 115, 121, 137, 146, 116, 118, 139, 117, 113, 112, 142, 136, 120, 109, 134, 135, 147, 119]

In [54]:
from datetime import date

# Get today's date in YYYY-MM-DD format
days_offset = 0
today = (date.today() + pd.DateOffset(days=days_offset)).strftime('%Y-%m-%d')

# Get today's schedule
games = statsapi.schedule(start_date=today, end_date=today)

# Loop through games and extract probable pitchers
for game in games:
    home_team = game['home_name']
    away_team = game['away_name']
    home_pitcher = game.get('home_probable_pitcher', 'TBD')
    away_pitcher = game.get('away_probable_pitcher', 'TBD')
    
    print(f"{away_team} ({away_pitcher}) @ {home_team} ({home_pitcher})")

Colorado Rockies (Germán Márquez) @ Miami Marlins (Max Meyer)
Los Angeles Angels (Tyler Anderson) @ Boston Red Sox (Richard Fitts)
Milwaukee Brewers (Aaron Civale) @ Cincinnati Reds (Brady Singer)
Detroit Tigers (Jack Flaherty) @ Chicago White Sox (Jonathan Cannon)
San Diego Padres (Stephen Kolek) @ San Francisco Giants (Logan Webb)
Minnesota Twins (Joe Ryan) @ Athletics (Luis Severino)
New York Mets (Paul Blackburn) @ Los Angeles Dodgers (Dustin May)


In [55]:
def spray_angle(hc_x, hc_y, stand):
    mult = 1
    if stand == 'L':
        mult = -1
    return mult * np.degrees(np.arctan2((hc_x - 125.42), (198.27 - hc_y)))

In [56]:
import os
api_key = os.getenv("api_key")
print(api_key)

fa1d935c1d67073458a1f970bc434075


In [57]:
# Note that BBE = attempts = number of events
pitcher_barrels = statcast_pitcher_exitvelo_barrels(2025)

In [58]:
start_date = (date.today() + pd.DateOffset(days=-30)).strftime('%Y-%m-%d')
end_date = today
pitcher_ids = get_starting_ids(days_offset=days_offset)
pitcher_id_nos = [p[0] for p in pitcher_ids]
starters_data = {}
pitchers_arsenals = {}
pitcher_opponents = {}
pitcher_games = {}
for id, opposing_team, game_id in pitcher_ids:
    pitcher_data = statcast_pitcher(start_dt=start_date, end_dt=end_date,player_id=id)
    pitcher_opponents[id] = opposing_team
    starters_data[id] = pitcher_data.copy()
    pitcher_games[id] = game_id
    pitchers_arsenals[id] = {'R': get_pitch_arsenal(pitcher_data, 'R'), 'L': get_pitch_arsenal(pitcher_data, 'L')}

['Max Meyer', 'Germán Márquez', 'Richard Fitts', 'Tyler Anderson', 'Brady Singer', 'Aaron Civale', 'Jonathan Cannon', 'Jack Flaherty', 'Logan Webb', 'Stephen Kolek', 'Luis Severino', 'Joe Ryan', 'Dustin May', 'Paul Blackburn']
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [59]:
# pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['attempts'] > 50]
pitcher_barrels_copy = pitcher_barrels[pitcher_barrels['player_id'].isin(pitcher_id_nos)].copy()
pitcher_barrels_copy['barrel_percentage'] = pitcher_barrels_copy['player_id'].apply(get_barrel_percentage, start_date=start_date, end_date=end_date)
pitcher_barrels_copy['left_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['left_pct'])
pitcher_barrels_copy['right_barrel_pct'] = pitcher_barrels_copy['barrel_percentage'].apply(lambda x: x['right_pct'])
pitcher_barrels_copy['opponent'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_opponents[x])
pitcher_barrels_copy['game_id'] = pitcher_barrels_copy['player_id'].apply(lambda x: pitcher_games.get(x))
pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'opponent']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,opponent
2,"Severino, Luis",622663,777669,207,9,4.30,"{'left_barrels': 1, 'right_barrels': 2, 'total...",0.03,0.04,away
21,"Webb, Logan",657277,777676,185,9,4.90,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.08,0.04,away
35,"Cannon, Jonathan",686563,777674,175,17,9.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.15,away
46,"Márquez, Germán",608566,777673,170,8,4.70,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.04,0.12,home
49,"Anderson, Tyler",542881,777670,167,13,7.80,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.06,0.08,home
70,"Meyer, Max",676974,777673,157,14,8.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.12,0.10,away
74,"Singer, Brady",663903,777671,152,18,11.80,"{'left_barrels': 9, 'right_barrels': 2, 'total...",0.20,0.04,away
90,"May, Dustin",669160,777675,143,11,7.70,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.18,away
103,"Ryan, Joe",657746,777669,138,20,14.50,"{'left_barrels': 5, 'right_barrels': 4, 'total...",0.14,0.24,home
104,"Flaherty, Jack",656427,777674,137,15,10.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.13,0.18,home


In [60]:
filtered_pitcher_barrels_copy = pitcher_barrels_copy[(pitcher_barrels_copy['left_barrel_pct'] > 0.07) | (pitcher_barrels_copy['right_barrel_pct'] > 0.07)].copy()
filtered_pitcher_barrels_copy['sides'] = filtered_pitcher_barrels_copy.apply(lambda row: get_sides(row['left_barrel_pct'], row['right_barrel_pct']),axis=1)
filtered_pitcher_barrels_copy['arsenal'] = filtered_pitcher_barrels_copy.apply(lambda row: pitchers_arsenals[row['player_id']], axis=1)
filtered_pitcher_barrels_copy[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent']]

,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent
21,"Webb, Logan",657277,777676,185,9,4.90,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.08,0.04,[L],"{'R': {'SI': 43.43434343434343, 'ST': 29.62962...",away
35,"Cannon, Jonathan",686563,777674,175,17,9.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.15,[R],"{'R': {'SI': 35.83815028901734, 'ST': 27.74566...",away
46,"Márquez, Germán",608566,777673,170,8,4.70,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.04,0.12,[R],"{'R': {'KC': 30.0, 'SI': 27.647058823529413, '...",home
49,"Anderson, Tyler",542881,777670,167,13,7.80,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.06,0.08,[R],"{'R': {'FF': 43.15068493150685, 'CH': 39.04109...",home
70,"Meyer, Max",676974,777673,157,14,8.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.12,0.10,"[L, R]","{'R': {'SI': 27.11111111111111, 'SL': 23.11111...",away
74,"Singer, Brady",663903,777671,152,18,11.80,"{'left_barrels': 9, 'right_barrels': 2, 'total...",0.20,0.04,[L],"{'R': {'SI': 50.0, 'SL': 32.075471698113205}, ...",away
90,"May, Dustin",669160,777675,143,11,7.70,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.18,"[L, R]","{'R': {'SI': 46.42857142857143, 'ST': 46.42857...",away
103,"Ryan, Joe",657746,777669,138,20,14.50,"{'left_barrels': 5, 'right_barrels': 4, 'total...",0.14,0.24,"[L, R]","{'R': {'FF': 48.148148148148145, 'ST': 19.2592...",home
104,"Flaherty, Jack",656427,777674,137,15,10.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.13,0.18,"[L, R]","{'R': {'FF': 46.15384615384615, 'KC': 26.92307...",home
176,"Kolek, Stephen",663568,777676,74,3,4.10,"{'left_barrels': 4, 'right_barrels': 1, 'total...",0.09,0.02,[L],"{'R': {'SI': 38.53658536585366, 'FF': 22.43902...",home


Next step is
- For each batter in a lineup that a pitcher is facing, we want to get their barrel % against the pitcher's arsenal (the pitchers > 18%)
- Calculate the weighted pull air percentage (pull % * air %)
- Also eventually can add blast %, iso

In [61]:
pitcher_df = filtered_pitcher_barrels_copy.copy()
pitcher_df['opposing_lineup'] = pitcher_df.apply(lambda row: get_opposing_lineup(row['game_id'], row['opponent']), axis=1)
pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup']]

Error fetching lineup for game 777676: list index out of range
Error fetching lineup for game 777669: list index out of range
Error fetching lineup for game 777676: list index out of range


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup
21,"Webb, Logan",657277,777676,185,9,4.90,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.08,0.04,[L],"{'R': {'SI': 43.43434343434343, 'ST': 29.62962...",away,None
35,"Cannon, Jonathan",686563,777674,175,17,9.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.15,[R],"{'R': {'SI': 35.83815028901734, 'ST': 27.74566...",away,"[678009, 650402, 681481, 682985, 693307, 69099..."
46,"Márquez, Germán",608566,777673,170,8,4.70,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.04,0.12,[R],"{'R': {'KC': 30.0, 'SI': 27.647058823529413, '...",home,"[669364, 660821, 682663, 669065, 672640, 67657..."
49,"Anderson, Tyler",542881,777670,167,13,7.80,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.06,0.08,[R],"{'R': {'FF': 43.15068493150685, 'CH': 39.04109...",home,"[680776, 646240, 608701, 665966, 663853, 64735..."
70,"Meyer, Max",676974,777673,157,14,8.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.12,0.10,"[L, R]","{'R': {'SI': 27.11111111111111, 'SL': 23.11111...",away,"[656541, 678662, 696100, 642731, 641857, 68666..."
74,"Singer, Brady",663903,777671,152,18,11.80,"{'left_barrels': 9, 'right_barrels': 2, 'total...",0.20,0.04,[L],"{'R': {'SI': 50.0, 'SL': 32.075471698113205}, ...",away,"[668930, 694192, 661388, 592885, 656555, 68621..."
90,"May, Dustin",669160,777675,143,11,7.70,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.18,"[L, R]","{'R': {'SI': 46.42857142857143, 'ST': 46.42857...",away,"[596019, 607043, 665742, 624413, 676724, 68314..."
103,"Ryan, Joe",657746,777669,138,20,14.50,"{'left_barrels': 5, 'right_barrels': 4, 'total...",0.14,0.24,"[L, R]","{'R': {'FF': 48.148148148148145, 'ST': 19.2592...",home,None
104,"Flaherty, Jack",656427,777674,137,15,10.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.13,0.18,"[L, R]","{'R': {'FF': 46.15384615384615, 'KC': 26.92307...",home,"[805367, 643565, 678246, 643217, 672820, 64194..."
176,"Kolek, Stephen",663568,777676,74,3,4.10,"{'left_barrels': 4, 'right_barrels': 1, 'total...",0.09,0.02,[L],"{'R': {'SI': 38.53658536585366, 'FF': 22.43902...",home,None


In [62]:
hitter_barrels = statcast_batter_exitvelo_barrels(2025)

In [63]:
def get_hitter_barrels(hitter_barrels, hitter_ids, start_date, end_date, pitch_mix, sides):
    if hitter_ids is None or len(hitter_ids) == 0:
        return {}
    hitter_barrels_copy = hitter_barrels[hitter_barrels['player_id'].isin(hitter_ids)].copy()
    hitter_barrels_copy['barrel_data'] = hitter_barrels_copy['player_id'].apply(hitter_barrel_percentage, start_date=start_date, end_date=end_date, pitch_mix=pitch_mix, sides=sides)
    hitter_barrels_copy['barrel_percentage'] = hitter_barrels_copy.apply(lambda row: get_barrel_pcts(row['barrel_data']), axis=1)
    hitter_barrels_copy['barrel_qual'] = hitter_barrels_copy['barrel_percentage'].apply(lambda x: check_barrel_pcts(x))
    # print(hitter_barrels_copy['barrel_qual'])
    filtered = hitter_barrels_copy[hitter_barrels_copy['barrel_qual'].apply(lambda x: x != [0,0])].copy()
    # print(filtered)
    if len(filtered) > 3:
        # Create a new column with just the first element
        filtered['barrel_qual_first'] = filtered['barrel_qual'].apply(lambda x: x[0])
        # Sort by that column and get top 3
        top_3 = filtered.nlargest(3, 'barrel_qual_first')
        # Drop the temporary column if you don't need it
        top_3 = top_3.drop('barrel_qual_first', axis=1)
        return dict(zip(top_3['last_name, first_name'], top_3['barrel_qual']))
    else:
        return dict(zip(filtered['last_name, first_name'], filtered['barrel_qual']))

In [64]:
hitter_pitcher_df = pitcher_df.copy()
hitter_pitcher_df['hitter_barrels'] = hitter_pitcher_df.apply(lambda row: get_hitter_barrels(hitter_barrels, row['opposing_lineup'], start_date, end_date, row['arsenal'], row['sides']), axis=1)
hitter_pitcher_df['num_hitters'] = hitter_pitcher_df['hitter_barrels'].apply(lambda x: len(x))
hitter_pitcher_df['hitter_team'] = hitter_pitcher_df.apply(lambda row: get_team(row['game_id'], row['opponent']), axis=1)
hitter_pitcher_df['start_time'] = hitter_pitcher_df['game_id'].apply(lambda x: get_start_time(x))
hitter_pitcher_df[['last_name, first_name','player_id','game_id','attempts','barrels','brl_percent','barrel_percentage', 'left_barrel_pct', 'right_barrel_pct', 'sides', 'arsenal', 'opponent', 'opposing_lineup', 'hitter_barrels', 'num_hitters', 'hitter_team', 'start_time']]

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


,"last_name, first_name",player_id,game_id,attempts,barrels,brl_percent,barrel_percentage,left_barrel_pct,right_barrel_pct,sides,arsenal,opponent,opposing_lineup,hitter_barrels,num_hitters,hitter_team,start_time
21,"Webb, Logan",657277,777676,185,9,4.90,"{'left_barrels': 3, 'right_barrels': 2, 'total...",0.08,0.04,[L],"{'R': {'SI': 43.43434343434343, 'ST': 29.62962...",away,None,{},0,Padres,
35,"Cannon, Jonathan",686563,777674,175,17,9.70,"{'left_barrels': 3, 'right_barrels': 5, 'total...",0.07,0.15,[R],"{'R': {'SI': 35.83815028901734, 'ST': 27.74566...",away,"[678009, 650402, 681481, 682985, 693307, 69099...","{'Torres, Gleyber': [0.1506849315068493, 0.301...",1,Tigers,6:40 pm.
46,"Márquez, Germán",608566,777673,170,8,4.70,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.04,0.12,[R],"{'R': {'KC': 30.0, 'SI': 27.647058823529413, '...",home,"[669364, 660821, 682663, 669065, 672640, 67657...","{'Norby, Connor': [0.1076923076923077, 0.24615...",1,Marlins,6:40 pm.
49,"Anderson, Tyler",542881,777670,167,13,7.80,"{'left_barrels': 2, 'right_barrels': 5, 'total...",0.06,0.08,[R],"{'R': {'FF': 43.15068493150685, 'CH': 39.04109...",home,"[680776, 646240, 608701, 665966, 663853, 64735...","{'Story, Trevor': [0.14285714285714285, 0.2678...",3,Red Sox,7:10 pm.
70,"Meyer, Max",676974,777673,157,14,8.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.12,0.10,"[L, R]","{'R': {'SI': 27.11111111111111, 'SL': 23.11111...",away,"[656541, 678662, 696100, 642731, 641857, 68666...","{'Goodman, Hunter': [0.11842105263157894, 0.35...",3,Rockies,6:40 pm.
74,"Singer, Brady",663903,777671,152,18,11.80,"{'left_barrels': 9, 'right_barrels': 2, 'total...",0.20,0.04,[L],"{'R': {'SI': 50.0, 'SL': 32.075471698113205}, ...",away,"[668930, 694192, 661388, 592885, 656555, 68621...",{},0,Brewers,7:10 pm.
90,"May, Dustin",669160,777675,143,11,7.70,"{'left_barrels': 4, 'right_barrels': 5, 'total...",0.09,0.18,"[L, R]","{'R': {'SI': 46.42857142857143, 'ST': 46.42857...",away,"[596019, 607043, 665742, 624413, 676724, 68314...","{'Lindor, Francisco': [0.12, 0.24], 'Soto, Jua...",3,Mets,7:10 pm.
103,"Ryan, Joe",657746,777669,138,20,14.50,"{'left_barrels': 5, 'right_barrels': 4, 'total...",0.14,0.24,"[L, R]","{'R': {'FF': 48.148148148148145, 'ST': 19.2592...",home,None,{},0,Athletics,7:05 pm.
104,"Flaherty, Jack",656427,777674,137,15,10.90,"{'left_barrels': 5, 'right_barrels': 5, 'total...",0.13,0.18,"[L, R]","{'R': {'FF': 46.15384615384615, 'KC': 26.92307...",home,"[805367, 643565, 678246, 643217, 672820, 64194...","{'Vargas, Miguel': [0.14457831325301204, 0.277...",1,White Sox,6:40 pm.
176,"Kolek, Stephen",663568,777676,74,3,4.10,"{'left_barrels': 4, 'right_barrels': 1, 'total...",0.09,0.02,[L],"{'R': {'SI': 38.53658536585366, 'FF': 22.43902...",home,None,{},0,Giants,


In [65]:
# Barrel percent, blast percent
all_keys = {}
for index, row in hitter_pitcher_df.iterrows():
    for key in row['hitter_barrels']:
        all_keys[(row['hitter_team'], key)] = row['hitter_barrels'][key]
sorted_batters = all_keys.items()
sorted_batters = sorted(sorted_batters, key=lambda x: x[1], reverse=True)
for batter in sorted_batters:
    print(batter)

(('Mets', 'Soto, Juan'), [np.float64(0.2361111111111111), np.float64(0.4444444444444444)])
(('Mets', 'Alonso, Pete'), [np.float64(0.18055555555555555), np.float64(0.3888888888888889)])
(('Rockies', 'McMahon, Ryan'), [np.float64(0.16923076923076924), np.float64(0.5076923076923077)])
(('Tigers', 'Torres, Gleyber'), [np.float64(0.1506849315068493), np.float64(0.3013698630136986)])
(('White Sox', 'Vargas, Miguel'), [np.float64(0.14457831325301204), np.float64(0.27710843373493976)])
(('Red Sox', 'Narváez, Carlos'), [np.float64(0.14285714285714285), np.float64(0.40816326530612246)])
(('Red Sox', 'Story, Trevor'), [np.float64(0.14285714285714285), np.float64(0.26785714285714285)])
(('Red Sox', 'Rafaela, Ceddanne'), [np.float64(0.1267605633802817), np.float64(0.323943661971831)])
(('Mets', 'Lindor, Francisco'), [np.float64(0.12), np.float64(0.24)])
(('Rockies', 'Goodman, Hunter'), [np.float64(0.11842105263157894), np.float64(0.35526315789473684)])
(('Rockies', 'Doyle, Brenton'), [np.float64(0.

To do:
- Add ISO